# 第三章 分组

# 笔记部分

In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv('data/table.csv',index_col='ID')
df.head()

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1103,S_1,C_1,M,street_2,186,82,87.2,B+
1104,S_1,C_1,F,street_2,167,81,80.4,B-
1105,S_1,C_1,F,street_4,159,64,84.8,B+


## 一. SAC过程

### 1. 内涵

SAC指的是分组操作中的split-apply-combine过程。

其中split指基于某一些规则，将数据拆成若干组，apply是指对每一组独立地使用函数，combine指将每一组的结果组合成某一类数据结构

### 2. apply过程

在该过程中，我们实际往往会遇到四类问题：

整合（Aggregation）——即分组计算统计量（如求均值、求每组元素个数）

变换（Transformation）——即分组对每个单元的数据进行操作（如元素标准化）

过滤（Filtration）——即按照某些规则筛选出一些组（如选出组内某一指标小于50的组）

综合问题——即前面提及的三种问题的混合

## 二、groupby函数

### 1. 分组函数的基本内容：

#### a）根据某一列分组

In [8]:
grouped_single = df.groupby('School')

经过groupby后会生成一个groupby对象，该对象本身不会返回任何东西，只有当相应的方法被调用才会起作用

例如取出某一个组

In [9]:
grouped_single.get_group('S_1').head()

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1103,S_1,C_1,M,street_2,186,82,87.2,B+
1104,S_1,C_1,F,street_2,167,81,80.4,B-
1105,S_1,C_1,F,street_4,159,64,84.8,B+


#### b）根据某几列分组

In [12]:
grouped_mul = df.groupby(['School','Class'])
grouped_mul.get_group(('S_1','C_1'))

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1103,S_1,C_1,M,street_2,186,82,87.2,B+
1104,S_1,C_1,F,street_2,167,81,80.4,B-
1105,S_1,C_1,F,street_4,159,64,84.8,B+


#### c）组容量与组数

In [13]:
grouped_single.size()

School
S_1    15
S_2    20
dtype: int64

In [14]:
grouped_mul.size()

School  Class
S_1     C_1      5
        C_2      5
        C_3      5
S_2     C_1      5
        C_2      5
        C_3      5
        C_4      5
dtype: int64

In [15]:
type(grouped_single)

pandas.core.groupby.generic.DataFrameGroupBy

In [17]:
grouped_mul.ngroups

7

#### d）组的遍历

In [18]:
for name,group in grouped_single:
    print(name)
    display(group.head())

S_1


,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1103,S_1,C_1,M,street_2,186,82,87.2,B+
1104,S_1,C_1,F,street_2,167,81,80.4,B-
1105,S_1,C_1,F,street_4,159,64,84.8,B+


S_2


,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
2101,S_2,C_1,M,street_7,174,84,83.3,C
2102,S_2,C_1,F,street_6,161,61,50.6,B+
2103,S_2,C_1,M,street_4,157,61,52.5,B-
2104,S_2,C_1,F,street_5,159,97,72.2,B+
2105,S_2,C_1,M,street_4,170,81,34.2,A


#### e）level参数（用于多级索引）和axis参数

In [25]:
# 不懂level和axis的具体用法
df.set_index(['Gender','School']).groupby(level=1, axis=0).get_group('S_1').head()

Class   Address  Height  Weight  Math Physics
Gender School                                              
M      S_1      C_1  street_1     173      63  34.0      A+
F      S_1      C_1  street_2     192      73  32.5      B+
M      S_1      C_1  street_2     186      82  87.2      B+
F      S_1      C_1  street_2     167      81  80.4      B-
       S_1      C_1  street_4     159      64  84.8      B+

In [32]:
# 可见level是指多层索引中的层次，axis是指跨行处理
df.set_index(['Gender','School']).groupby(level=0,axis=0).get_group('M').head()

Class   Address  Height  Weight  Math Physics
Gender School                                              
M      S_1      C_1  street_1     173      63  34.0      A+
       S_1      C_1  street_2     186      82  87.2      B+
       S_1      C_2  street_5     188      68  97.0      A-
       S_1      C_2  street_6     160      53  58.8      A+
       S_1      C_3  street_4     161      68  31.5      B+

### 2. groupby对象的特点

#### a）查看所有可调用的方法

由此可见，groupby对象可以使用相当多的函数，灵活程度很高

In [28]:
dir(grouped_single)
print([attr for attr in dir(grouped_single) if not attr.startswith('_')])

['Address', 'Class', 'Gender', 'Height', 'Math', 'Physics', 'School', 'Weight', 'agg', 'aggregate', 'all', 'any', 'apply', 'backfill', 'bfill', 'boxplot', 'corr', 'corrwith', 'count', 'cov', 'cumcount', 'cummax', 'cummin', 'cumprod', 'cumsum', 'describe', 'diff', 'dtypes', 'expanding', 'ffill', 'fillna', 'filter', 'first', 'get_group', 'groups', 'head', 'hist', 'idxmax', 'idxmin', 'indices', 'last', 'mad', 'max', 'mean', 'median', 'min', 'ndim', 'ngroup', 'ngroups', 'nth', 'nunique', 'ohlc', 'pad', 'pct_change', 'pipe', 'plot', 'prod', 'quantile', 'rank', 'resample', 'rolling', 'sem', 'shift', 'size', 'skew', 'std', 'sum', 'tail', 'take', 'transform', 'tshift', 'var']


#### b）分组对象的head和first

#### 对分组对象使用head函数，返回的是每个组的前几行，而不是数据集前几行

first显示的是以分组为索引的每组的第一个分组信息

In [30]:
grouped_single.first()

,Class,Gender,Address,Height,Weight,Math,Physics
School,,,,,,,
S_1,C_1,M,street_1,173,63,34.0,A+
S_2,C_1,M,street_7,174,84,83.3,C


#### c) 分组依据

对于groupby函数而言，分组的依据是非常自由的，只要是与数据框长度相同的列表即可，同时支持函数型分组

In [34]:
df.groupby(np.random.choice(['a','b','c'],df.shape[0])).get_group('a').head()
#相当于将np.random.choice(['a','b','c'],df.shape[0])当做新的一列进行分组

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1104,S_1,C_1,F,street_2,167,81,80.4,B-
1203,S_1,C_2,M,street_6,160,53,58.8,A+
1204,S_1,C_2,F,street_5,162,63,33.8,B
1304,S_1,C_3,M,street_2,195,70,85.2,A


从原理上说，我们可以看到利用函数时，传入的对象就是索引，因此根据这一特性可以做一些复杂的操作

In [38]:
df[:5].groupby(lambda x:print(x)).head(3)

1101
1102
1103
1104
1105


,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1103,S_1,C_1,M,street_2,186,82,87.2,B+


根据奇偶行分组

In [39]:
df.groupby(lambda x:'奇数行' if not df.index.get_loc(x)%2==1 else '偶数行').groups


{'偶数行': Int64Index([1102, 1104, 1201, 1203, 1205, 1302, 1304, 2101, 2103, 2105, 2202,
             2204, 2301, 2303, 2305, 2402, 2404],
            dtype='int64', name='ID'),
 '奇数行': Int64Index([1101, 1103, 1105, 1202, 1204, 1301, 1303, 1305, 2102, 2104, 2201,
             2203, 2205, 2302, 2304, 2401, 2403, 2405],
            dtype='int64', name='ID')}

In [41]:
df.index.get_loc(1101)

0

#### 如果是多层索引，那么lambda表达式中的输入就是元组，下面实现的功能为查看两所学校中男女生分别均分是否及格

注意：此处只是演示groupby的用法，实际操作不会这样写

In [43]:
math_score = df.set_index(['Gender','School'])['Math'].sort_index()
grouped_score = df.set_index(['Gender','School']).sort_index().\
            groupby(lambda x:(x,'均分及格' if math_score[x].mean()>=60 else '均分不及格'))
for name,_ in grouped_score:print(name)

(('F', 'S_1'), '均分及格')
(('F', 'S_2'), '均分及格')
(('M', 'S_1'), '均分及格')
(('M', 'S_2'), '均分不及格')


In [46]:
math_score = df.set_index(['Gender','School'])['Math'].sort_index()
math_score.head()

Gender  School
F       S_1       32.5
        S_1       80.4
        S_1       84.8
        S_1       63.5
        S_1       33.8
Name: Math, dtype: float64

#### d）groupby的[]操作

可以用[]选出groupby对象的某个或者某几个列，上面的均分比较可以如下简洁地写出：

In [48]:
df.groupby(['Gender','School'])['Math'].mean()>=60

Gender  School
F       S_1        True
        S_2        True
M       S_1        True
        S_2       False
Name: Math, dtype: bool

#### 用列表可选出多个属性列：

In [52]:
df.groupby(['Gender','School'])[['Math','Height']].mean()

Math      Height
Gender School                       
F      S_1     64.100000  173.125000
       S_2     66.427273  173.727273
M      S_1     63.342857  178.714286
       S_2     51.155556  172.000000

#### e）连续型变量分组

例如利用cut函数对数学成绩分组：

In [53]:
bins = [0,40,60,80,90,100]
cuts = pd.cut(df['Math'],bins=bins) #可选label添加自定义标签
df.groupby(cuts)['Math'].count()

Math
(0, 40]       7
(40, 60]     10
(60, 80]      9
(80, 90]      7
(90, 100]     2
Name: Math, dtype: int64

In [55]:
type(cuts)

pandas.core.series.Series

## 三、聚合、过滤和变换

### 1.聚合

所谓聚合就是把一堆数，变成一个标量，因此mean/sum/size/count/std/var/sem/describe/first/last/nth/min/max都是聚合函数

In [60]:
group_m = grouped_single['Math']
group_m.std().values/np.sqrt(group_m.count().values)== group_m.sem().values

array([ True,  True])

#### b)同时使用多个聚合函数

In [61]:
group_m.agg(['sum','mean','std'])

,sum,mean,std
School,,,
S_1,956.2,63.746667,23.077474
S_2,1191.1,59.555000,17.589305


利用元祖进行重命名

In [63]:
group_m.agg([('rename_sum','sum'),('rename_mean','mean')])

,rename_sum,rename_mean
School,,
S_1,956.2,63.746667
S_2,1191.1,59.555000


指定哪些函数作用哪些列 （字典）

In [64]:
grouped_mul.agg({'Math':['mean','max'],'Height':'var'})

Math       Height
               mean   max    var
School Class                    
S_1    C_1    63.78  87.2  183.3
       C_2    64.30  97.0  132.8
       C_3    63.16  87.7  179.2
S_2    C_1    58.56  83.3   54.7
       C_2    62.80  85.4  256.0
       C_3    63.06  95.5  205.7
       C_4    53.80  67.7  300.2